In [ ]:
# Imports

import os
# Disable some chatty warnings from Tensorflow:
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '1' 

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import numpy as np
import tensorflow as tf
import keras_tuner as kt
import random
import pandas as pd
import sys

from credentials import CONNECTION_INFO
from constants import *
from crackers import Caesar_Cracker

import encoders
import db_connect
import helpers
import tf_helpers
import models

In [ ]:
CAESAR_KEY_MODEL = models.load_model(CAESAR_KEY_MODEL_PATH)
CAESAR_TEXT_MODEL = models.load_model(CAESAR_TEXT_MODEL_PATH)
SUBSTITUTION_KEY_MODEL = None
SUBSTITUTION_TEXT_MODEL = models.load_model(SUBSTITUTION_TEXT_MODEL_PATH)